In [34]:
import pandas as pd
import requests
from pandas_datareader import data as pdr
import yfinance as yf

In [51]:
source_file = "data.xlsx" # исходник данных для получения universe
start = '2017-01-01' # дата начало получения для finance.yahoo.com
end = '2021-09-04' # дата окончания получения для finance.yahoo.com


In [53]:
main_data = pd.read_excel(source_file, index_col=None, engine="openpyxl", parse_dates=True, sheet_name = "universe")

naufor_df = main_data[main_data['source']=="naufor.ru"]
naufor_codes = naufor_df['code'].to_numpy().tolist()

yfinance_df = main_data[main_data['source']=="finance.yahoo.com"]
yfinance_codes = yfinance_df['code'].to_numpy().tolist()


# Функции экстракции

In [65]:
def get_pif_naufor_quotes(code):
    '''
    получаем данные с naufor.ru
    '''
    referer = 'http://pif.naufor.ru/pif.asp?act=view&id={}'.format(code)
    url = 'http://pif.naufor.ru/pif.asp?act=getpayschajson&id={}&begindate=20030313&enddate=20210601'.format(code)
    resp = requests.get(url, headers={'referer': referer})
    
    blank_array = []
    result_dict = resp.json()
    
    for index, date in enumerate(result_dict['Label']): 
        dd = date.split('.')
        blank_array.append({
            'source': 'naufor.ru',
            'code': code,
            'date': "{year}-{month}-{day}".format(year=dd[2], month=dd[1], day=dd[0]),
            'price': result_dict['UnitValue'][index]
        })

    return blank_array

def get_yahoo_finance_quote(ticker, Start, End):
    '''
    получаем данные с finance.yahoo.com
    '''
    yf.pdr_override() 
    data2 = pdr.get_data_yahoo(ticker, start=Start, end=End)
    output = pd.DataFrame(data=data2['Close'])
    output['code'] = ticker
    clr = output.reset_index().to_dict('records')
    res = []
    for row in clr:
        res.append({
            'source': 'finance.yahoo.com',
            'code': ticker,
            'date': row['Date'].strftime('%Y-%m-%d'),
            'price': row['Close']
        })
    return res

# Извлечение данных с naufor

In [48]:
all_data = []
for code in naufor_codes:
    all_data += get_pif_naufor_quotes(code)
    print("[naufor]Ready code: {}".format(code))

Ready code: 299
Ready code: 60
Ready code: 2746
Ready code: 4017
Ready code: 2880
Ready code: 651
Ready code: 3830
Ready code: 662
Ready code: 525
Ready code: 63
Ready code: 2322
Ready code: 3823
Ready code: 4394
Ready code: 66
Ready code: 606
Ready code: 256
Ready code: 107


In [49]:
for code in yfinance_codes:
    all_data += get_yahoo_finance_quote(code, start, end)
    print("[get_yahoo_finance_quote]Ready code: {}".format(code))

[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: GAZP.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: GMKN.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: LKOH.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: MGNT.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: MTSS.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: NLMK.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: NVTK.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: PLZL.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_

In [66]:
df = pd.DataFrame(all_data)
df.to_excel("tickers_prices.xlsx", index=False)